In [14]:
#  Functions to find all the Julia files in a directory and subdirectories
function filter_jl_files(dir_list::Vector{String})
    """Filter a list of filenames into a list of Julia files and a list of non-Julia files"""
    jl_files = filter(x -> endswith(x, ".jl"), dir_list)
    non_jl_files = setdiff(dir_list, jl_files)
    return jl_files, non_jl_files
end

function add_filepaths!(filepaths::Dict{String, String}, dir_path::String, dir_list::Vector{String})
    """Add a list of filepaths to a dictionary"""
    for file in dir_list
        filepaths[file] = joinpath(dir_path, file)
    end
end

function find_jl_files(dir_path::String)
    """Find all Julia files in a directory and subdirectories"""
    filepaths = Dict{String, String}()
    jl_files, non_jl_files = filter_jl_files(readdir(dir_path))
    add_filepaths!(filepaths, dir_path, jl_files)
    new_dirs = filter(x -> isdir(joinpath(dir_path, x)), non_jl_files)
    for new_dir in new_dirs
        merge!(filepaths, find_jl_files(joinpath(dir_path, new_dir)))
    end
    return filepaths
end

find_jl_files (generic function with 1 method)

In [29]:
# Functions to read a Julia file, clean it, and return it as a long string
function clean_file(dir_path::String)
    """
        filestring = clean_file(dir_path::String)

    Read a Julia file line by line,
    removing all comments and doc strings,
    then appending each line to a string with a space
    """
    open(dir_path, "r") do file
        filestring = ""
        long_comment = false
        for line in eachline(file)
            # Check for doc strings and ignore them
            if contains(line, """\"\"\"""")
                long_comment = !long_comment
            end
            if long_comment
                continue
            else 
                # Remove comments
                if !startswith(strip(line), "#") && line != ""
                    # Remove tabs
                    line = replace(line, "\t" => " ")
                    # Make all one line, but add space to avoid concatenating words
                    filestring = string(filestring, " ", line)
                end
            end
        end
        return filestring
    end
end

function make_filestrings(filepath::Dict{String, String})
    """Make a dictionary of filestrings from a dictionary of filepaths"""
    filestrings = Dict{String, String}()
    for (filename, filepath) in filepaths
        filestrings[filename] = clean_file(filepath)
    end
    return filestrings
end

make_filestrings (generic function with 2 methods)

In [16]:
# Functions to extract model features from a filestring
function extract_feature_name(filestring::AbstractString, filter_prefix::AbstractString)
    """Extract the name of a model feature from a file-substring"""
    x = split(filestring, ",")
    # Try to extract the name based on a prefix
    x2 = split(x[2], " ")
    x2 = filter(x -> startswith(x, filter_prefix), x2)
    if !isempty(x2)
        x = x2[1]
    else
        x = x[2]
    end
    # Remove any indexing
    if contains(x, "[")
        x = split(x, "[")[1]
    end
    # Remove whitespace
    return strip(x)
end

function filter_filestring(filestring::String, filter_string::String)
    """Create a list of the names of all the model feature of a given type, from a filestring"""
    cases = split(filestring, filter_string)
    feature_names = Vector{String}()
    filter_prefix = string(filter_string[2])
    for i in 2:length(cases)
        push!(feature_names, extract_feature_name(cases[i], filter_prefix))
    end
    return feature_names
end

filter_filestring (generic function with 1 method)

In [57]:
# Get all Julia files in the src directory
src_path = joinpath(dirname(pwd()), "src")
filepaths = find_jl_files(src_path)

# Clean all Julia files and return them as strings
filestrings = make_filestrings(filepaths)

exp_and_vars = Dict("var" => Dict{String, Any}(), "exp" => Dict{String, Any}())

# Create a dictionary of all the model features we'd like to search for, and the details of interest
features_to_search = Dict(
    "var" => Dict("filter_string" => "@variable", "details" => ["created", "accessed"]),
    "exp" => Dict("filter_string" => "@expression", "details" => ["created", "accessed"]),
)

# Create dictionary to hold search results
# We could be more specific and make the last Dict{String, String||List}, but this is safer
search_results = Dict{String, Dict{String, Any}}()
for (search_name, search_details) in features_to_search
    search_results[search_name] = Dict{String, Dict{String, Any}}()
end

# Find the file in which each feature is created
for (search_name, search_details) in features_to_search
    find_feature_creation!(search_results[search_name], filestrings, search_details["filter_string"])
end


In [50]:
function find_feature_creation!(feature_results::Dict{String, Any}, filestrings::Dict{String, String}, feature_name::String)
    """Find the files in which a set of features is created"""
    for (filename, filestring) in filestrings
        new_feature_names = filter_filestring(filestring, feature_name)
        for new_name in new_feature_names
            if !haskey(feature_results, new_name)
                feature_results[new_name] = Dict{String, Any}()
            end
            feature_results[new_name]["created"] = filename
        end
    end
end

find_feature_creation! (generic function with 1 method)

In [17]:
function find_exp_var_created(filepaths::Dict{String, String})
    # exp_and_vars = Dict{String, Dict{String, String}}("var" => Dict{String, String}(), "exp" => Dict{String, String}())
    exp_and_vars = Dict("var" => Dict{String, Any}(), "exp" => Dict{String, Any}())
    for (filename, filepath) in filepaths
        # println(filename)
        filestring = clean_file(filepath)
        new_var = filter_filestring(filestring, "@variable")
        for var in new_var
            if !haskey(exp_and_vars["var"], var)
                exp_and_vars["var"][var] = Dict("created" => filename, "accessed" => Vector{String}())
            end
            exp_and_vars["var"][var]["created"] = filename
        end
        new_exp = filter_filestring(filestring, "@expression")
        for exp in new_exp
            if !haskey(exp_and_vars["exp"], exp)
                exp_and_vars["exp"][exp] = Dict("created" => filename, "accessed" => Vector{String}())
            end
            exp_and_vars["exp"][exp]["created"] = filename
        end
    end
    return exp_and_vars
end

find_exp_var_created (generic function with 1 method)

In [18]:
function check_access(filepaths::Dict{String, String}, target_names::Vector{String})
    file_accesses = Dict{String, Vector{String}}()
    for (filename, filepath) in filepaths
        filestring = clean_file(filepath)
        file_accesses[filename] = Vector{String}()
        for target_name in target_names
            if contains(filestring, target_name)
                push!(file_accesses[filename], target_name)
            end
        end
    end
    return file_accesses
end

function push_accesses!(exp_or_vars::Dict{String, Any}, accesses::Dict{String, Vector{String}})
    for (filename, model_names) in accesses
        for model_name in model_names
            push!(exp_or_vars[model_name]["accessed"], filename)
        end
    end
end

function get_all_accesses!(exp_and_vars::Dict{String, Dict{String, Any}}, filepaths::Dict{String, String})
    var_accessed = check_access(filepaths, collect(keys(exp_and_vars["var"])))
    push_accesses!(exp_and_vars["var"], var_accessed)
    exp_accessed = check_access(filepaths, collect(keys(exp_and_vars["exp"])))
    push_accesses!(exp_and_vars["exp"], exp_accessed)
end

check_access (generic function with 1 method)

In [21]:
function make_table_header(headers::Set{String}, model_type::String)
    fun_headers = copy(headers)
    table_header = "|$(model_type) name"
    table_divider = "|:-"
    if "created" in fun_headers
        table_header = string(table_header, "|created")
        table_divider = string(table_divider, "|:-")
        delete!(fun_headers, "created")
    end    
    for header in fun_headers
        table_header = string(table_header, "|", header)
        table_divider = string(table_divider, "|", ":-")
    end
    table_header = string(table_header, "|")
    table_divider = string(table_divider, "|")
    return table_header, table_divider
end

make_table_header (generic function with 1 method)

In [22]:
function make_table_row(model_name::String, model_dict::Dict{String, Any}, headers::Set{String})
    table_row = "|$(model_name)"
    fun_headers = copy(headers)
    if "created" in fun_headers
        table_row = string(table_row, "|", model_dict["created"])
        delete!(fun_headers, "created")
    end
    for header in fun_headers
        table_row = string(table_row, "|", join(model_dict[header], ", "))
    end
    table_row = string(table_row, "|")
    return table_row
end

make_table_row (generic function with 1 method)

In [23]:
function data_2_markdown(exp_or_vars::Dict{String, Any}, model_type::String, headers::Set{String})
    table = Vector{String}()
    push!(table, "## $(model_type)s")
    table_header, table_divider = make_table_header(headers, model_type)
    push!(table, table_header)
    push!(table, table_divider)
    sorted_names = sort(collect(keys(exp_or_vars)))
    # for (name, data) in exp_or_vars
    for name in sorted_names
        push!(table, make_table_row(name, exp_or_vars[name], headers))
    end
    return table
end

data_2_markdown (generic function with 1 method)

In [38]:
# Get all Julia files in the src directory
src_path = joinpath(dirname(pwd()), "src")
filepaths = find_jl_files(src_path)

# Find where all the Variables and Expressions are defined
exp_and_vars = find_exp_var_created(filepaths)

# Find where each Variable is accessed
get_all_accesses!(exp_and_vars, filepaths)
headers = Set{String}(["created", "accessed"])

var_table = data_2_markdown(exp_and_vars["var"], "Variable", headers)
exp_table = data_2_markdown(exp_and_vars["exp"], "Expression", headers)

open(joinpath(pwd(), "model_table.md"), "w") do io
    println(io, "# Model Variables and Expressions")
    for line in var_table
        println(io, line)
    end
    for line in exp_table
        println(io, line)
    end
end

# Find where the Objective is defined, and each of the components of the Objective

In [43]:
exp_and_vars["var"]["vREG_discharge"]

Dict{String, Any} with 2 entries:
  "accessed" => ["storage_symmetric.jl", "reserves.jl", "storage_all.jl"]
  "created"  => "reserves.jl"

In [25]:
filename = "thermal_commit.jl"
filepath = filepaths[filename]
filestring = clean_file(filepath)
q = split(filestring, "function ")
q = q[2:end]
for w in q
    # w = q[1]
    w = split(w, "(")
    fun_name = w[1]
    fun_args = split(split(w[2], ")")[1], ", ")
    println(fun_name)
    for a in fun_args
        println(a)
    end
end

thermal_commit
EP::Model
inputs::Dict
Reserves::Int
thermal_commit_reserves
EP::Model
inputs::Dict
hoursbefore
p::Int
t::Int
b::UnitRange{Int}


In [26]:
function find_functions(filepaths::Dict{String, String})
    function_names = Dict{String, String}()
    for (filename, filepath) in filepaths
        filestring = clean_file(filepath)
        new_functions = filter_filestring(filestring, "function ")
        for fun in new_functions
            if !haskey(function_names, fun)
                function_names[fun] = filename
            end
        end
    end
    return function_names
end

find_functions (generic function with 1 method)